# Capstone 3: Data Capture
<b>PROJECT GOAL</b><br>
The goal of this project is to understand and predict the valuation of publicly-traded companies as measured by market capitalization (the target or response variable).  Financial statement data will be used as predictor variables (features) to assess whether regression models of these fundamental variables can predicted future market capitalization.<br>

<b>DATA</b><br>
All data are sourced from Alpha Vantage via their API (https://www.alphavantage.co/documentation/).

<i>response variable (target)</i><br>
'mkt_cap' = [number of shares outstanding] x [share price]<br><br>
<i>predictor variables (features)</i><br>
'rev' = abc<br>
'rev_gth_yoy' = abc<br>
'rev_gth_qoq' = abc<br>
'rev_accel_yoy' = abc<br>
list to be completed

<b>DATA CAPTURE NOTEBOOK</b><br>
The purpose of this notebook is to:
<ol type="1">
  <li>define the population of data to be analyzed</li>
  <li>load relevant financial data sets from Alpha Vantage</li>
  <li>define and create meta data that specifies the basic structure of the data for analysis</li>
  <li>store the captured data in JSON format to facilitate downstream analyses</li>
</ol> 

## 1 Import Python Modules

The item api_av_call is a script with custom functions to call the Alpha Vantage API.

In [1]:
import api_av_call as av_api     #script with custom functions for API calls

from importlib import reload 

import pandas as pd
import numpy as np

import datetime as dttm
from dateutil.relativedelta import relativedelta
import time

import requests
import json
import pickle

## 2 Load: Security Status
Load a list of securities by stock ticker of active, U.S.-listed stocks as of Aug 31, 2021, convert date data types, save to JSON.

<b>API KEY</b>

In [2]:
#API key
reload(av_api)

API_KEY_AV = ''
f01 = open('../config.txt', 'r')
API_KEY_AV = f01.readline()
f01.close()

<b>LOAD ACTIVE SECURITIES</b>

The Alpha Vantage API returns a CSV file.  The custom function av_api.call_listingstatus calls the API, captures the CSV file and returns a dataframe.

In [3]:
#API parameters
statusdate01 = dttm.date(2021,8,31)     #as of date
state01 = 'active' #active,delisted

#API Call
#df_sec_status = av_api.call_listingstatus(statusdate01,state01,API_KEY_AV)

In [ ]:
df_sec_status.loc[:,'ipodate'] = pd.to_datetime(df_sec_status.loc[:,'ipodate'])
df_sec_status.loc[:,'delistingdate'] = pd.to_datetime(df_sec_status.loc[:,'delistingdate'])
df_sec_status.loc[:,'status_date'] = pd.to_datetime(df_sec_status.loc[:,'status_date'])
df_sec_status.loc[:,'etl_timestamp'] = pd.to_datetime(df_sec_status.loc[:,'etl_timestamp'])

In [40]:
#save: df_sec_status
#json
filnm11 = '../data/sec_status_' + statusdate11.strftime('%Y%m%d') + '.json'
df_sec_status_11 = df_sec_status.copy()
df_sec_status_11.loc[:,'ipodate'] = df_sec_status_11.loc[:,'ipodate'].dt.strftime('%Y-%m-%d')
df_sec_status_11.loc[:,'delistingdate'] = df_sec_status_11.loc[:,'delistingdate'].dt.strftime('%Y-%m-%d')
df_sec_status_11.loc[:,'status_date'] = df_sec_status_11.loc[:,'status_date'].dt.strftime('%Y-%m-%d')
df_sec_status_11.loc[:,'etl_timestamp'] = df_sec_status_11.loc[:,'etl_timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
#df_sec_status_1.to_json(filnm11)   

Load data from JSON file, if data are already captured.

In [4]:
#load if already captured: df_sec_status
#json
filnm12 = '../data/sec_status_' + statusdate01.strftime('%Y%m%d') + '.json'
with open(filnm12) as f12:
    sec_status_dict12 = json.load(f12)

df_sec_status = pd.DataFrame.from_dict(sec_status_dict12)

In [5]:
df_sec_status.loc[:,'ipodate'] = pd.to_datetime(df_sec_status.loc[:,'ipodate'])
df_sec_status.loc[:,'delistingdate'] = pd.to_datetime(df_sec_status.loc[:,'delistingdate'])
df_sec_status.loc[:,'status_date'] = pd.to_datetime(df_sec_status.loc[:,'status_date'])
df_sec_status.loc[:,'etl_timestamp'] = pd.to_datetime(df_sec_status.loc[:,'etl_timestamp'])

In [6]:
df_sec_status.info()
df_sec_status.head()

<class 'pandas.core.frame.DataFrame'>
Index: 11458 entries, 0 to 11457
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ticker         11458 non-null  object        
 1   name           11394 non-null  object        
 2   exchange       11458 non-null  object        
 3   assettype      11458 non-null  object        
 4   ipodate        11458 non-null  datetime64[ns]
 5   delistingdate  0 non-null      datetime64[ns]
 6   status         11458 non-null  object        
 7   status_date    11458 non-null  datetime64[ns]
 8   etl_timestamp  11458 non-null  datetime64[ns]
dtypes: datetime64[ns](4), object(5)
memory usage: 895.2+ KB


,ticker,name,exchange,assettype,ipodate,delistingdate,status,status_date,etl_timestamp
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaT,Active,2021-08-31,2021-09-12 15:15:24
1,AA,Alcoa Corp,NYSE,Stock,2016-11-01,NaT,Active,2021-08-31,2021-09-12 15:15:24
2,AAA,AAF First Priority CLO Bond ETF,NYSE ARCA,ETF,2020-09-09,NaT,Active,2021-08-31,2021-09-12 15:15:24
3,AAAU,Goldman Sachs Physical Gold ETF,NYSE ARCA,ETF,2018-08-15,NaT,Active,2021-08-31,2021-09-12 15:15:24
4,AAC,Ares Acquisition Corporation - Class A,NYSE,Stock,2021-03-25,NaT,Active,2021-08-31,2021-09-12 15:15:24


## 3 Load: Security Overview
Using the list of active securities from above, loop individual stocks to capture security overview data (a list of attributes by stock ticker), convert float and date data types, save to JSON.  The Alpha Vantage API returns a JSON file.  The function av_api.call_av_api calls the API, captures the JSON string and returns a dataframe.  

Below is a list of different API parameters used for data capture

<b> API Calls - API Function, Attribute List </b>

'OVERVIEW'<br>
['Symbol','CIK','Currency','Country','Sector','Industry','MarketCapitalization','DividendPerShare',
    'ShortPercentFloat','PercentInsiders']

'EARNINGS'<br>
['fiscalDateEnding','reportedDate','reportedEPS','estimatedEPS','surprise','surprisePercentage']

'BALANCE_SHEET'<br>
['fiscalDateEnding','reportedCurrency','cashAndShortTermInvestments','deferredRevenue',
    'capitalLeaseObligations','longTermDebt','commonStockSharesOutstanding']
    
'INCOME_STATEMENT'<br>
['fiscalDateEnding','reportedCurrency','totalRevenue','grossProfit',
    'operatingExpenses','ebitda','netIncome']
    
'CASH_FLOW'<br>
['fiscalDateEnding','reportedCurrency',
    'proceedsFromIssuanceOfCommonStock','paymentsForRepurchaseOfCommonStock','capitalExpenditures',
    'dividendPayoutCommonStock',
    'operatingCashflow','cashflowFromInvestment','cashflowFromFinancing','changeInCashAndCashEquivalents']
    
'TIME_SERIES_DAILY_ADJUSTED'<br>
['1. open','2. high','3. low','4. close','5. adjusted close','6. volume','7. dividend amount',
    '8. split coefficient']

'TIME_SERIES_MONTHLY_ADJUSTED'<br>
['1. open','2. high','3. low','4. close','5. adjusted close','6. volume','7. dividend amount']

<b>CAPTURE SECURITY OVERVIEW DATA</b>

In [51]:
#GET SECURITY OVERVIEW DATA
#API parameters
apifunc02 = 'OVERVIEW'
otptsz02 = 'none'
attr_lst02 = ['Symbol','CIK','Currency','Country','Sector','Industry','MarketCapitalization','DividendPerShare',
    'ShortPercentFloat','PercentInsiders']
tmout02 = 2.85
ntry02 = 3
sleepmin02 = 0.35

#Result sets
is_stock02 = df_sec_status.loc[:,'assettype'] == 'Stock'
df_sec_oview = pd.DataFrame()
df_sec_oview_excp = pd.DataFrame()

#Loop symbols
ct02 = 0
resp_ct02 = 0 
excp_ct02 = 0 
for idx02, row02 in df_sec_status.loc[is_stock02,:].loc[:,:].iterrows():
    
    #get data
    ct02 = ct02 + 1
    tickn02 = row02[0]
    df_res_obj02 = av_api.call_av_api(tickn02,apifunc02,otptsz02,API_KEY_AV,attr_lst02,tmout02,sleepmin02,ntry02)

    #merge to create security overview dataframe
    if df_res_obj02[0] == 0:
        resp_ct02 = resp_ct02 + 1
        if resp_ct02 == 1:
            df_sec_oview = df_res_obj02[1]
        else:
            df_sec_oview = pd.concat([df_sec_oview,df_res_obj02[1]],ignore_index=True)
    else:
        excp_ct02 = excp_ct02 + 1
        print('exception:',ct02,tickn02,row02[3])
        if excp_ct02 == 1:
            df_sec_oview_excp = df_res_obj02[2]
        else:
            df_sec_oview_excp = pd.concat([df_sec_oview_excp,df_res_obj02[2]],ignore_index=True)  
    
    #0.8 sec per request min (max 75 req per min)
    time.sleep(0.85)
    if ct02 % 50 == 0:
        print('iteration:',ct02,tickn02,row02[3]) 


iteration: 1 A Stock
iteration: 2 AA Stock
iteration: 3 AAC Stock
iteration: 4 AAC-U Stock
iteration: 5 AAC-WS Stock
iteration: 6 AACG Stock
iteration: 7 AACIU Stock
iteration: 8 AADI Stock


In [12]:
#assign data types (assume: MC NaN, others 0.0)
isnone_mc02 = df_sec_oview.loc[:,'MarketCapitalization'] == 'None'
df_sec_oview.loc[isnone_mc02,'MarketCapitalization'] = np.NaN
df_sec_oview.loc[:,'MarketCapitalization'] = df_sec_oview.loc[:,'MarketCapitalization'].astype({'MarketCapitalization':'float64'})  

isnone_div02 = df_sec_oview.loc[:,'DividendPerShare'] == 'None'
df_sec_oview.loc[isnone_div02,'DividendPerShare'] = 0.0
df_sec_oview.loc[:,'DividendPerShare'] = df_sec_oview.loc[:,'DividendPerShare'].astype({'DividendPerShare':'float64'})  

isnone_sht02 = df_sec_oview.loc[:,'ShortPercentFloat'] == 'None'
df_sec_oview.loc[isnone_sht02,'ShortPercentFloat'] = 0.0
df_sec_oview.loc[:,'ShortPercentFloat'] = df_sec_oview.loc[:,'ShortPercentFloat'].astype({'ShortPercentFloat':'float64'})  

isnone_ins02 = df_sec_oview.loc[:,'PercentInsiders'] == 'None'
df_sec_oview.loc[isnone_ins02,'PercentInsiders'] = 0.0
df_sec_oview.loc[:,'PercentInsiders'] = df_sec_oview.loc[:,'PercentInsiders'].astype({'PercentInsiders':'float64'})  

df_sec_oview.loc[:,'etl_timestamp'] = df_sec_oview.loc[:,'etl_timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [16]:
#save: df_sec_oview, df_sec_oview_excp
#json
filnm21 = '../data/sec_overview_' + statusdate01.strftime('%Y%m%d') + '.json'
filnm22 = '../data/sec_overview_excp_' + statusdate01.strftime('%Y%m%d') + '.json'
df_sec_oview_21 = df_sec_oview.copy()
df_sec_oview_excp_22 = df_sec_oview_excp.copy()

df_sec_oview_21.to_json(filnm21)
df_sec_oview_excp_22.to_json(filnm22)

Load data from JSON file, if data are already captured.

In [82]:
#load if already captured: : df_sec_oview, df_sec_oview_excp
#json
filnm23 = '../data/sec_oview_' + statusdate01.strftime('%Y%m%d') + '.json'
filnm24 = '../data/sec_oview_excp_' + statusdate01.strftime('%Y%m%d') + '.json'
with open(filnm23) as f23:
    sec_oview_dict23 = json.load(f23)
with open(filnm24) as f24:
    sec_oview_excp_dict24 = json.load(f24)
        
df_sec_oview = pd.DataFrame.from_dict(sec_oview_dict23)
df_sec_oview_excp = pd.DataFrame.from_dict(sec_oview_excp_dict24)

df_sec_oview.loc[:,'etl_timestamp'] = pd.to_datetime(df_sec_oview.loc[:,'etl_timestamp'])

In [83]:
df_sec_oview.info()
df_sec_oview.head()

<class 'pandas.core.frame.DataFrame'>
Index: 8026 entries, 0 to 8025
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Symbol                8026 non-null   object        
 1   CIK                   8026 non-null   object        
 2   Currency              8026 non-null   object        
 3   Country               8026 non-null   object        
 4   Sector                8026 non-null   object        
 5   Industry              8026 non-null   object        
 6   MarketCapitalization  6543 non-null   float64       
 7   DividendPerShare      8026 non-null   float64       
 8   ShortPercentFloat     8026 non-null   float64       
 9   PercentInsiders       8026 non-null   float64       
 10  etl_timestamp         8026 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(4), object(6)
memory usage: 752.4+ KB


,Symbol,CIK,Currency,Country,Sector,Industry,MarketCapitalization,DividendPerShare,ShortPercentFloat,PercentInsiders,etl_timestamp
0,A,1090872,USD,USA,LIFE SCIENCES,INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY ...,5.365160e+10,0.762,0.0129,0.266,2021-09-12 17:21:04
1,AA,1675149,USD,USA,MANUFACTURING,PRIMARY PRODUCTION OF ALUMINUM,9.249867e+09,0.360,0.0664,0.961,2021-09-12 17:21:05
2,AAC,1829432,USD,USA,REAL ESTATE & CONSTRUCTION,BLANK CHECKS,1.220000e+09,0.000,0.0005,0.000,2021-09-12 17:21:06
3,AAC-U,1829432,USD,USA,REAL ESTATE & CONSTRUCTION,BLANK CHECKS,NaN,0.000,0.0000,0.000,2021-09-12 17:21:07
4,AAC-WS,1829432,USD,USA,REAL ESTATE & CONSTRUCTION,BLANK CHECKS,NaN,0.000,0.0000,0.000,2021-09-12 17:21:10


## 4 Load: Security Financial Data
Using the list of active securities enriched with overview data, rank by market capitalization, flag stocks with $1m (million) market capitalization or larger, and loop individual stocks to capture security financial data.  Then convert date data types and save to JSON.  The Alpha Vantage API returns a JSON file.  The function av_api.call_av_api calls the API, captures the JSON string and returns a dataframe.

In this section, EARNINGS, BALANCE_SHEET, INCOME_STATEMENT, and CASH_FLOW data are captured from Alpha Vantage.

<b>RANK BY MARKET CAPITALIZATION</b>

In [6]:
#RANK BY MARKET CAP
df_sec_oview.loc[:,'mktcap_rk'] = df_sec_oview.loc[:,'MarketCapitalization'].rank(axis=0,ascending=False)
#subset market cap >= cutoff
mktcap_ctff = 10**9
is_gte_mktcap_ctff = ( df_sec_oview.loc[:,'MarketCapitalization'] >= mktcap_ctff ) & ( ~df_sec_oview.loc[:,'MarketCapitalization'].isna() )
df_sec_oview.sort_values(by='mktcap_rk',inplace=True)

df_sec_oview_mktcap = df_sec_oview.loc[is_gte_mktcap_ctff,:].copy()
df_sec_oview_mktcap.reset_index(drop=True,inplace=True)   

In [7]:
 df_sec_oview_mktcap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2970 entries, 0 to 2969
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Symbol                2970 non-null   object        
 1   CIK                   2970 non-null   object        
 2   Currency              2970 non-null   object        
 3   Country               2970 non-null   object        
 4   Sector                2970 non-null   object        
 5   Industry              2970 non-null   object        
 6   MarketCapitalization  2970 non-null   float64       
 7   DividendPerShare      2970 non-null   float64       
 8   ShortPercentFloat     2970 non-null   float64       
 9   PercentInsiders       2970 non-null   float64       
 10  etl_timestamp         2970 non-null   datetime64[ns]
 11  mktcap_rk             2970 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(6)
memory usage: 278.6+ KB


In [8]:
#top 10 
df_sec_oview_mktcap.head(10)

,Symbol,CIK,Currency,Country,Sector,Industry,MarketCapitalization,DividendPerShare,ShortPercentFloat,PercentInsiders,etl_timestamp,mktcap_rk
0,AAPL,320193,USD,USA,TECHNOLOGY,ELECTRONIC COMPUTERS,2.462504e+12,0.835,0.0056,0.071,2021-09-12 17:21:30,1.0
1,MSFT,789019,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,2.222228e+12,2.240,0.0056,0.078,2021-09-12 20:29:06,2.0
2,GOOG,1652044,USD,USA,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",1.885287e+12,0.000,0.0045,0.043,2021-09-12 19:27:21,3.5
3,GOOGL,1652044,USD,USA,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",1.885287e+12,0.000,0.0052,0.148,2021-09-12 19:27:23,3.5
4,AMZN,1018724,USD,USA,TRADE & SERVICES,RETAIL-CATALOG & MAIL-ORDER HOUSES,1.756920e+12,0.000,0.0112,13.610,2021-09-12 17:30:47,5.0
5,FB,1326801,USD,USA,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",1.067694e+12,0.000,0.0123,0.602,2021-09-12 19:14:36,6.0
6,TSLA,1318605,USD,USA,MANUFACTURING,MOTOR VEHICLES & PASSENGER CAR BODIES,7.375732e+11,0.000,0.0335,18.960,2021-09-12 21:29:42,7.0
7,TSM,1046179,USD,USA,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,6.377323e+11,10.500,0.0035,0.006,2021-09-12 21:29:44,8.0
8,BRK-B,1067983,USD,USA,FINANCE,"FIRE, MARINE & CASUALTY INSURANCE",6.285030e+11,0.000,0.0040,0.184,2021-09-12 17:45:42,9.0
9,BRK-A,1067983,USD,USA,FINANCE,"FIRE, MARINE & CASUALTY INSURANCE",6.285025e+11,0.000,0.0002,42.240,2021-09-12 17:45:41,10.0


In [9]:
#bottom 10
df_sec_oview_mktcap.tail(10)

,Symbol,CIK,Currency,Country,Sector,Industry,MarketCapitalization,DividendPerShare,ShortPercentFloat,PercentInsiders,etl_timestamp,mktcap_rk
2960,ELMS,1784168,USD,USA,REAL ESTATE & CONSTRUCTION,BLANK CHECKS,1.012060e+09,0.000,0.0451,54.190,2021-09-12 19:09:38,2961.0
2961,NTST,1798100,USD,USA,REAL ESTATE & CONSTRUCTION,REAL ESTATE INVESTMENT TRUSTS,1.011470e+09,0.700,0.0338,0.229,2021-09-12 20:37:47,2962.0
2962,MASS,1555279,USD,USA,LIFE SCIENCES,"MEASURING & CONTROLLING DEVICES, NEC",1.009469e+09,0.000,0.1350,26.970,2021-09-12 20:22:11,2963.0
2963,NXGN,708818,USD,USA,TECHNOLOGY,SERVICES-COMPUTER INTEGRATED SYSTEMS DESIGN,1.008983e+09,0.000,0.0250,19.360,2021-09-12 20:38:47,2964.0
2964,QNST,1117297,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",1.006682e+09,0.000,0.0205,8.790,2021-09-12 20:58:55,2965.0
2965,RBCAA,921557,USD,USA,FINANCE,STATE COMMERCIAL BANKS,1.006482e+09,1.188,0.0099,50.150,2021-09-12 21:00:11,2966.0
2966,LPI,1528129,USD,USA,ENERGY & TRANSPORTATION,CRUDE PETROLEUM & NATURAL GAS,1.003174e+09,0.000,0.1580,2.776,2021-09-12 20:19:52,2967.0
2967,RPT,842183,USD,USA,REAL ESTATE & CONSTRUCTION,REAL ESTATE INVESTMENT TRUSTS,1.000868e+09,0.150,0.0136,1.629,2021-09-12 21:05:05,2968.0
2968,VRAY,1597313,USD,USA,LIFE SCIENCES,ELECTROMEDICAL & ELECTROTHERAPEUTIC APPARATUS,1.000709e+09,0.000,0.0840,2.990,2021-09-12 21:37:33,2969.0
2969,GGPI,1847127,USD,USA,REAL ESTATE & CONSTRUCTION,BLANK CHECKS,1.000000e+09,0.000,0.0000,0.000,2021-09-12 19:23:56,2970.0


<b>CAPTURE SECURITY FINANCIAL STATEMENT DATA</b>

In [226]:
#GET SECURITY DATA: FINANCIAL STATEMENTS
#clock start
load_start03 = dttm.datetime.now()

#API parameters
apifunc03 = 'CASH_FLOW'
otptsz03 = 'none'
attr_lst03 = ['fiscalDateEnding','reportedCurrency', 'proceedsFromIssuanceOfCommonStock',
    'paymentsForRepurchaseOfCommonStock','capitalExpenditures', 'dividendPayoutCommonStock', 
    'operatingCashflow','cashflowFromInvestment','cashflowFromFinancing','changeInCashAndCashEquivalents']
tmout03 = 2.85
sleepmin03 = 0.35
ntry03 = 3

#Date parameters
n_mthbk_st03 = 31
n_mthbk_ed03 = 0
refdate03 = dttm.date(2021,8,31)
end_refmth_firstday03 = refdate03.replace(day=1)
startdate03 = end_refmth_firstday03 - relativedelta(months=n_mthbk_st03)
enddate03 = end_refmth_firstday03 - dttm.timedelta(days=1) - relativedelta(months=(n_mthbk_ed03 - 1))

#Result sets
df_sec_res03 = pd.DataFrame()
df_sec_res_excp03 = pd.DataFrame()

#Loop symbols
ct03 = 0
resp_ct03 = 0 
excp_ct03 = 0 
for idx03, row03 in df_sec_oview_mktcap.loc[:,:].iterrows():
    
    #get data
    ct03 = ct03 + 1
    tickn03 = row03[0]
    df_res_obj03 = av_api.call_av_api(tickn03,apifunc03,otptsz03,API_KEY_AV,attr_lst03,tmout03,sleepmin03,ntry03)
    
    #security dataframe
    if df_res_obj03[0] == 0:
        resp_ct03 = resp_ct03 + 1
        #format response dataframe
        df_resp03 = df_res_obj03[1].copy()
        if apifunc03 == 'EARNINGS':
            df_resp03.loc[:,'reportedDate'] = pd.to_datetime(df_resp03.loc[:,'reportedDate'])
            df_resp03.loc[:,'fiscalDateEnding'] = pd.to_datetime(df_resp03.loc[:,'fiscalDateEnding'])
        elif apifunc03 in ['BALANCE_SHEET','INCOME_STATEMENT','CASH_FLOW']:
            df_resp03.loc[:,'fiscalDateEnding'] = pd.to_datetime(df_resp03.loc[:,'fiscalDateEnding'])

        df_resp03.loc[:,'Symbol'] = tickn03 
        colset03 = list(df_res_obj03[1].columns)
        colset03.insert(0, 'Symbol')
        df_resp03 = df_resp03.loc[:,colset03] 
        #define date range
        if apifunc03 == 'EARNINGS':
            is_daterng03 = ( (df_resp03.loc[:,'reportedDate'] >= startdate03) & 
                (df_resp03.loc[:,'reportedDate'] <= enddate03) )
        elif apifunc03 in ['BALANCE_SHEET','INCOME_STATEMENT','CASH_FLOW']:
            is_daterng03 = ( (df_resp03.loc[:,'fiscalDateEnding'] >= startdate03) & 
                (df_resp03.loc[:,'fiscalDateEnding'] <= enddate03) )
        if resp_ct03 == 1:
            df_sec_res03 = df_resp03.loc[is_daterng03,:]
        else:
            df_sec_res03 = pd.concat([df_sec_res03,df_resp03.loc[is_daterng03,:]],ignore_index=True)
    else:
        excp_ct03 = excp_ct03 + 1
        print('exception:',ct03,tickn03,row03[4])
        if excp_ct03 == 1:
            df_sec_res_excp03 = df_res_obj03[2]
        else:
            df_sec_res_excp03 = pd.concat([df_sec_res_excp03,df_res_obj03[2]],ignore_index=True)  
    
    #wait sec per request 
    time.sleep(0.83)
    if ct03 % 30 == 0:
        print('iteration:',ct03,tickn03,row03[4]) 

#clock end
load_end03 = dttm.datetime.now()
load_elapsed03 = load_end03 - load_start03 
print(load_start03,load_end03,load_elapsed03)

iteration: 30 CRM TECHNOLOGY
iteration: 60 TXN MANUFACTURING
iteration: 90 PDD TRADE & SERVICES
iteration: 120 ABNB TRADE & SERVICES
iteration: 150 HCA LIFE SCIENCES
iteration: 180 REGN LIFE SCIENCES
iteration: 210 TWLO TECHNOLOGY
iteration: 240 CMG TRADE & SERVICES
iteration: 270 SCCO ENERGY & TRANSPORTATION
iteration: 300 MCHP MANUFACTURING
iteration: 330 APTV MANUFACTURING
iteration: 360 PPG LIFE SCIENCES
iteration: 390 EFX TRADE & SERVICES
iteration: 420 CHWY TRADE & SERVICES
iteration: 450 PCAR MANUFACTURING
iteration: 480 XP FINANCE
iteration: 510 AVTR LIFE SCIENCES
iteration: 540 Z TRADE & SERVICES
iteration: 570 KMX TRADE & SERVICES
iteration: 600 TECH LIFE SCIENCES
iteration: 630 STX TECHNOLOGY
iteration: 660 EXAS TRADE & SERVICES
iteration: 690 LYFT TRADE & SERVICES
iteration: 720 AES ENERGY & TRANSPORTATION
iteration: 750 PTC TECHNOLOGY
iteration: 780 NVCR LIFE SCIENCES
iteration: 810 DISCA TECHNOLOGY
iteration: 840 LNC FINANCE
iteration: 870 CHRW ENERGY & TRANSPORTATION
ite

In [229]:
#data types for dates
if apifunc03 == 'EARNINGS':
    df_sec_res03.loc[:,'fiscalDateEnding'] = pd.to_datetime(df_sec_res03.loc[:,'fiscalDateEnding'])
    df_sec_res03.loc[:,'reportedDate'] = pd.to_datetime(df_sec_res03.loc[:,'reportedDate'])
    df_sec_res03.loc[:,'etl_timestamp'] = pd.to_datetime(df_sec_res03.loc[:,'etl_timestamp'])
elif apifunc03 in ['BALANCE_SHEET','INCOME_STATEMENT','CASH_FLOW']:
    df_sec_res03.loc[:,'fiscalDateEnding'] = pd.to_datetime(df_sec_res03.loc[:,'fiscalDateEnding'])
    df_sec_res03.loc[:,'etl_timestamp'] = pd.to_datetime(df_sec_res03.loc[:,'etl_timestamp'])

In [242]:
df_sec_res03.info()
df_sec_res03.head(22)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27684 entries, 0 to 27683
Data columns (total 12 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Symbol                              27684 non-null  object        
 1   fiscalDateEnding                    27684 non-null  datetime64[ns]
 2   reportedCurrency                    27684 non-null  object        
 3   proceedsFromIssuanceOfCommonStock   27684 non-null  object        
 4   paymentsForRepurchaseOfCommonStock  27684 non-null  object        
 5   capitalExpenditures                 27684 non-null  object        
 6   dividendPayoutCommonStock           27684 non-null  object        
 7   operatingCashflow                   27684 non-null  object        
 8   cashflowFromInvestment              27684 non-null  object        
 9   cashflowFromFinancing               27684 non-null  object        
 10  changeInCashAndCashEqu

,Symbol,fiscalDateEnding,reportedCurrency,proceedsFromIssuanceOfCommonStock,paymentsForRepurchaseOfCommonStock,capitalExpenditures,dividendPayoutCommonStock,operatingCashflow,cashflowFromInvestment,cashflowFromFinancing,changeInCashAndCashEquivalents,etl_timestamp
0,AAPL,2021-06-30,USD,0,22900000000,2093000000,3767000000,21094000000,3572000000,-29396000000,21744000000,2021-09-27 22:47:51.980114
1,AAPL,2021-03-31,USD,561000000,18548000000,2269000000,3447000000,23981000000,-10368000000,-11326000000,23630000000,2021-09-27 22:47:51.980114
2,AAPL,2020-12-31,USD,0,24775000000,3500000000,3613000000,38763000000,-8584000000,-32249000000,-2070000000,2021-09-27 22:47:51.980114
3,AAPL,2020-09-30,USD,450000000,17187000000,1784000000,3511000000,20576000000,5531000000,-21357000000,24689000000,2021-09-27 22:47:51.980114
4,AAPL,2020-06-30,USD,0,15891000000,1565000000,3656000000,16271000000,-5165000000,-19116000000,10865000000,2021-09-27 22:47:51.980114
5,AAPL,2020-03-31,USD,428000000,18574000000,1853000000,3375000000,13311000000,9013000000,-20940000000,10896000000,2021-09-27 22:47:51.980114
6,AAPL,2019-12-31,USD,2000000,20706000000,2107000000,3539000000,30516000000,-13668000000,-25407000000,-8559000000,2021-09-27 22:47:51.980114
7,AAPL,2019-09-30,USD,390000000,17444000000,2777000000,3479000000,19910000000,-798000000,-21039000000,24313000000,2021-09-27 22:47:51.980114
8,AAPL,2019-06-30,USD,1000000,16955000000,2000000000,3629000000,11636000000,27502000000,-26804000000,10044000000,2021-09-27 22:47:51.980114
9,AAPL,2019-03-31,USD,390000000,23702000000,2363000000,3443000000,11155000000,13348000000,-29457000000,11561000000,2021-09-27 22:47:51.980114


In [236]:
#save 'EARNINGS', 'BALANCE_SHEET','INCOME_STATEMENT','CASH_FLOW'
data_elmt30 = 'cshflw'

#json
filnm31 = '../data/sec_' + data_elmt30 + '_' + statusdate01.strftime('%Y%m%d') + '.json'
filnm32 = '../data/sec_' + data_elmt30 + '_excp_' + statusdate01.strftime('%Y%m%d') + '.json'

df_sec_res_31 = df_sec_res03.copy()
df_sec_res_excp_32 = df_sec_res_excp03.copy()

if apifunc03 == 'EARNINGS':
    df_sec_res_31.loc[:,'fiscalDateEnding'] = df_sec_res_31.loc[:,'fiscalDateEnding'].dt.strftime('%Y-%m-%d')
    df_sec_res_31.loc[:,'reportedDate'] = df_sec_res_31.loc[:,'reportedDate'].dt.strftime('%Y-%m-%d')
    df_sec_res_31.loc[:,'etl_timestamp'] = df_sec_res_31.loc[:,'etl_timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
elif apifunc03 in ['BALANCE_SHEET','INCOME_STATEMENT','CASH_FLOW']:
    df_sec_res_31.loc[:,'fiscalDateEnding'] = df_sec_res_31.loc[:,'fiscalDateEnding'].dt.strftime('%Y-%m-%d')
    df_sec_res_31.loc[:,'etl_timestamp'] = df_sec_res_31.loc[:,'etl_timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_sec_res_31.to_json(filnm31)
df_sec_res_excp_32.to_json(filnm32)

Load data from JSON file, if data are already captured.

In [219]:
#load, if already captured 'EARNINGS', 'BALANCE_SHEET','INCOME_STATEMENT','CASH_FLOW'
data_elmt30 = 'earn'

#json - df_sec_res
filnm33 = '../data/sec_' + data_elmt30 + '_' + statusdate01.strftime('%Y%m%d') + '.json'
filnm34 = '../data/sec_' + data_elmt30 + '_excp_' + statusdate01.strftime('%Y%m%d') + '.json'

with open(filnm33) as f33:
    sec_res_dict33 = json.load(f33)
with open(filnm34) as f34:
    sec_res_excp_dict34 = json.load(f34)
        
df_sec_res_33 = pd.DataFrame.from_dict(sec_res_dict33)
df_sec_res_excp_34 = pd.DataFrame.from_dict(sec_res_excp_dict34)

if data_elmt30 == 'earn':
    df_sec_res_33.loc[:,'fiscalDateEnding'] = pd.to_datetime(df_sec_res_33.loc[:,'fiscalDateEnding'])
    df_sec_res_33.loc[:,'reportedDate'] = pd.to_datetime(df_sec_res_33.loc[:,'reportedDate'])
    df_sec_res_33.loc[:,'etl_timestamp'] = pd.to_datetime(df_sec_res_33.loc[:,'etl_timestamp'])
elif data_elmt30 in ['balsht','incstmt','cshflw']:
    df_sec_res_33.loc[:,'fiscalDateEnding'] = pd.to_datetime(df_sec_res_33.loc[:,'fiscalDateEnding'])
    df_sec_res_33.loc[:,'etl_timestamp'] = pd.to_datetime(df_sec_res_33.loc[:,'etl_timestamp'])

## 5 Load: Security Meta, Price Data

Meta data are defined and saved for each stock ticker.  These meta data define, for each stock ticker, the fiscal periods and associated financial reporting dates used to determine the values for the target variable (market capitalization) and the feature variables (various financial metrics).

Briefly, for a given stock ticker, the financial metrics for the fiscal period ending on date t are publicly reported on some later report date, s.  The subsequent fiscal period ending on date t+1 is reported on report date s+1.  For each report date, s, define a measurement date, u, where u is the latest date before report date s having an end-of-day stock price (i.e., the last date the stock market is open before report date s).

Given the above date convention, the regression problem is as follows:

<i>target variable</i> = market capitalization on measurement date u+1<br>
<i>feature variables</i> = financial metrics from fiscal period t communicated on report date s<br>
<i>prediction goal</i>: predict market capitalization on measurement date u+1 using feature variables from report date s

Again using the list of stocks with market captitalization of $1m or larger, meta data (fiscal, report, and measurement dates) are defined and saved for each stock ticker using dates from the EARNINGS and INCOME_STATEMENT API calls.  

Fiscal period and report dates are captured from the EARNINGS data.  However, fiscal period dates are also captured from the INCOME_STATEMENT data if these dates are not present in the EARNINGS data.  But the INCOME_STATEMENT lacks report dates, so in these cases report dates are estimated from fiscal period dates using an estimate for the interval between fiscal period date and report date.  For each stock ticker, this interval is estimated using the median interval observed for that stock ticker in the EARNINGS data with a floor of 20 days (20 days is about the 5th percentile over all tickers, which is 19 days).

Given these measurement dates for each stock ticker, end-of-day prices for these dates are captured from Alpha Vantage using API parameter TIME_SERIES_DAILY_ADJUSTED.  The Alpha Vantage API returns a JSON file.  The function av_api.call_av_api calls the API, captures the JSON string and returns a dataframe.

<b>INITIALIZE EMPTY DATAFRAME FOR META DATA</b>

In [36]:
#INITIAL META DATAFRAME
meta_coldict04 = { 'ticker': pd.Series([], dtype='str'),'mkt_cap_202108': pd.Series([], dtype='float'),
    'rpt_seq': pd.Series([], dtype='float'),'rpt_yr': pd.Series([], dtype='int'),
    'rpt_date_t': pd.Series([], dtype='datetime64[ns]'),'rpt_date_tp1': pd.Series([], dtype='datetime64[ns]'),
    'fisc_date_t': pd.Series([], dtype='datetime64[ns]'),'fisc_date_tp1': pd.Series([], dtype='datetime64[ns]'),
    'meas_date_t': pd.Series([], dtype='datetime64[ns]'),'meas_date_tp1': pd.Series([], dtype='datetime64[ns]'),
    'rd_t_estflg': pd.Series([], dtype='int'),'rd_tp1_estflg': pd.Series([], dtype='int') }

df_sec_meta04 = pd.DataFrame(meta_coldict04)
df_sec_meta04.info()
df_sec_meta04.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ticker          0 non-null      object        
 1   mkt_cap_202108  0 non-null      float64       
 2   rpt_seq         0 non-null      float64       
 3   rpt_yr          0 non-null      int64         
 4   rpt_date_t      0 non-null      datetime64[ns]
 5   rpt_date_tp1    0 non-null      datetime64[ns]
 6   fisc_date_t     0 non-null      datetime64[ns]
 7   fisc_date_tp1   0 non-null      datetime64[ns]
 8   meas_date_t     0 non-null      datetime64[ns]
 9   meas_date_tp1   0 non-null      datetime64[ns]
 10  rd_t_estflg     0 non-null      int64         
 11  rd_tp1_estflg   0 non-null      int64         
dtypes: datetime64[ns](6), float64(2), int64(3), object(1)
memory usage: 124.0+ bytes


,ticker,mkt_cap_202108,rpt_seq,rpt_yr,rpt_date_t,rpt_date_tp1,fisc_date_t,fisc_date_tp1,meas_date_t,meas_date_tp1,rd_t_estflg,rd_tp1_estflg


<b>LOAD EARNINGS DATA</b>

In [11]:
#LOAD EARNINGS DATA
filnm41 = '../data/sec_earn_' + statusdate01.strftime('%Y%m%d') + '.json'

with open(filnm41) as f41:
    sec_earn_dict04 = json.load(f41)
        
df_sec_earn_04 = pd.DataFrame.from_dict(sec_earn_dict04)
df_sec_earn_04.loc[:,'fiscalDateEnding'] = pd.to_datetime(df_sec_earn_04.loc[:,'fiscalDateEnding'])
df_sec_earn_04.loc[:,'reportedDate'] = pd.to_datetime(df_sec_earn_04.loc[:,'reportedDate'])
df_sec_earn_04.loc[:,'etl_timestamp'] = pd.to_datetime(df_sec_earn_04.loc[:,'etl_timestamp'])

<b>INTERVAL BETWEEN FISCAL PERIOD AND REPORT DATES</b>

All Stocks: Median, mean, standard deviation, 5th percentile, 95th percentile

In [86]:
fiscrpt_diff_04 = (df_sec_earn_04.loc[:,'reportedDate'] - df_sec_earn_04.loc[:,'fiscalDateEnding'])

fiscrpt_diff_totmed04 = fiscrpt_diff_04.median().days
fiscrpt_diff_totmean04 = fiscrpt_diff_04.mean().days
fiscrpt_diff_totstd04 = fiscrpt_diff_04.std().days
fiscrpt_diff_tot05pctl = fiscrpt_diff_04.quantile(0.05).days
fiscrpt_diff_tot95pctl = fiscrpt_diff_04.quantile(0.90).days

print(fiscrpt_diff_totmed04,fiscrpt_diff_totmean04,fiscrpt_diff_totstd04,
      fiscrpt_diff_tot05pctl,fiscrpt_diff_tot95pctl)

36 40 20 19 84


<b>LOAD INCOME STATEMENT DATA</b>

In [12]:
#LOAD INCOME_STATEMENT DATA
filnm42 = '../data/sec_incstmt_' + statusdate01.strftime('%Y%m%d') + '.json'

with open(filnm42) as f42:
    sec_incstmt_dict04 = json.load(f42)
        
df_sec_incstmt_04 = pd.DataFrame.from_dict(sec_incstmt_dict04)
df_sec_incstmt_04.loc[:,'fiscalDateEnding'] = pd.to_datetime(df_sec_incstmt_04.loc[:,'fiscalDateEnding'])
df_sec_incstmt_04.loc[:,'etl_timestamp'] = pd.to_datetime(df_sec_incstmt_04.loc[:,'etl_timestamp'])

In [13]:
df_sec_incstmt_04.info()
df_sec_incstmt_04.head()

<class 'pandas.core.frame.DataFrame'>
Index: 27200 entries, 0 to 27199
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Symbol             27200 non-null  object        
 1   fiscalDateEnding   27200 non-null  datetime64[ns]
 2   reportedCurrency   27200 non-null  object        
 3   totalRevenue       27200 non-null  object        
 4   grossProfit        27200 non-null  object        
 5   operatingExpenses  27200 non-null  object        
 6   ebitda             27200 non-null  object        
 7   netIncome          27200 non-null  object        
 8   etl_timestamp      27200 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(7)
memory usage: 2.1+ MB


,Symbol,fiscalDateEnding,reportedCurrency,totalRevenue,grossProfit,operatingExpenses,ebitda,netIncome,etl_timestamp
0,AAPL,2021-06-30,USD,80769000000,35255000000,11129000000,27866000000,21744000000,2021-09-27 20:48:52
1,AAPL,2021-03-31,USD,88914000000,38079000000,10576000000,31478000000,23630000000,2021-09-27 20:48:52
2,AAPL,2020-12-31,USD,110801000000,44328000000,10794000000,36883000000,28755000000,2021-09-27 20:48:52
3,AAPL,2020-09-30,USD,64698000000,24689000000,9914000000,18237000000,12673000000,2021-09-27 20:48:52
4,AAPL,2020-06-30,USD,58988000000,22680000000,9589000000,16586000000,11253000000,2021-09-27 20:48:52


<b>CAPTURE SECURITY META AND END-OF-DAY PRICE DATA</b>

In [37]:
#GET SECURITY DATA: META, PRICE
#clock start
load_start04 = dttm.datetime.now()

#API parameters
apifunc04 = 'TIME_SERIES_DAILY_ADJUSTED'
otptsz04 = 'full'
attr_lst04 = ['1. open','2. high','3. low','4. close','5. adjusted close',
    '6. volume','7. dividend amount', '8. split coefficient']
tmout04 = 2.85
sleepmin04 = 0.35
ntry04 = 3

#Date parameters
n_mthbk_st04 = 31
n_mthbk_ed04 = 0
refdate04 = dttm.date(2021,8,31)
end_refmth_firstday04 = refdate04.replace(day=1)
startdate04 = end_refmth_firstday04 - relativedelta(months=n_mthbk_st04)
enddate04 = end_refmth_firstday04 - dttm.timedelta(days=1) - relativedelta(months=(n_mthbk_ed04 - 1))

#Result sets
fiscrpt_diff_flr04 = 20
n_samp04 = 5
df_sec_price_eod04 = pd.DataFrame()
df_sec_price_eod_excp04 = pd.DataFrame()

#Loop symbols
ct04 = 0
resp_ct04 = 0 
excp_ct04 = 0 
for idx04, row04 in df_sec_oview_mktcap.loc[:,:].iterrows():
    
    #get ticker
    ct04 = ct04 + 1
    tickn04 = row04[0]
    mkt_cap04 = row04[6]
    
    #(1) meta data: earnings dates from earn
    is_earn_tickn04 = df_sec_earn_04.loc[:,'Symbol'] == tickn04
    df_sec_earn_tmp04 = pd.DataFrame()
    df_sec_earn_tmp04 = df_sec_earn_04.loc[is_earn_tickn04,:].sort_values(by='reportedDate',ascending=False).copy()
    df_sec_earn_tmp04.reset_index(inplace=True,drop=True)
    #skip if too little earnings data
    if len(df_sec_earn_tmp04) < 2:
        print('exception:',ct04,tickn04,row04[4],'under 2 earnings records')
        continue
        
    #meta data - earn: list to insert
    lst_earn_insrt04 = []
    lst_earn_insrt04 = [ [ rcd041[0], mkt_cap04, 0, rcd041[2].year,
        rcd041[2], dttm.date(2000,1,1), rcd041[1], dttm.date(2000,1,1),
        dttm.date(2000,1,1), dttm.date(2000,1,1), 0, 0 ] 
        for idx041,rcd041 in df_sec_earn_tmp04.loc[1:n_samp04,:].iterrows() ]    
    n_rcd_earn04 = len(lst_earn_insrt04)
    
    #meta data - earn: next report, fiscal date  
    for idx042,val042 in enumerate(lst_earn_insrt04):
        lst_earn_insrt04[idx042][5] = df_sec_earn_tmp04.loc[idx042,'reportedDate'] 
        lst_earn_insrt04[idx042][7] = df_sec_earn_tmp04.loc[idx042,'fiscalDateEnding']   
        
    #meta data - earn: df from list
    df_meta_insrt04 = pd.DataFrame(lst_earn_insrt04)
    df_meta_insrt04.columns = list(df_sec_meta04.columns)    
    
    #(2) meta data: earnings dates from incstmt
    n_rcd_inc04 = 0
    if n_rcd_earn04 < n_samp04:
        #remaining samples needed, fiscal dates from earnings data
        n_rcd_inc04 = n_samp04 - n_rcd_earn04
        fisc_date_earn_max04 = df_meta_insrt04.loc[:,'fisc_date_t'].max()
        fisc_date_earn_set041 = set(df_meta_insrt04.loc[:,'fisc_date_t'].unique())
        fisc_date_earn_set042 = set(df_sec_earn_tmp04.loc[0:n_samp04,'fiscalDateEnding'].unique())

        #estimate fiscal to report interval, set to overal median if < 21 days
        fiscrpt_diff_med04 = np.floor((df_sec_earn_tmp04.loc[:,'reportedDate'] - 
            df_sec_earn_tmp04.loc[:,'fiscalDateEnding']).median().days)
        fiscrpt_diff_add04 = np.max([np.max([fiscrpt_diff_flr04,fiscrpt_diff_med04]),fiscrpt_diff_totmed04])

        #incstmt: ticker, <=enddate
        is_incstmt_tickn041 = ( (df_sec_incstmt_04.loc[:,'Symbol'] == tickn04) & 
            (df_sec_incstmt_04.loc[:,'fiscalDateEnding'] <= fisc_date_earn_max04) )
        df_sec_incstmt_tmp041 = pd.DataFrame()
        df_sec_incstmt_tmp041 = df_sec_incstmt_04.loc[is_incstmt_tickn041,:].sort_values(by='fiscalDateEnding',ascending=False).copy()
        df_sec_incstmt_tmp041.reset_index(inplace=True,drop=True)

        #incstmt: top5
        df_sec_incstmt_tmp042 = df_sec_incstmt_tmp041.loc[1:n_samp04,:].copy()

        #incstmt: not existing fiscal date from earnings data
        is_incstmt_tickn042 = ~df_sec_incstmt_tmp042.loc[:,'fiscalDateEnding'].isin(fisc_date_earn_set041)
        df_sec_incstmt_tmp043 = df_sec_incstmt_tmp042.loc[is_incstmt_tickn042,:].copy()
        df_sec_incstmt_tmp043.reset_index(inplace=True,drop=True)

        #remaining samples needed
        df_sec_incstmt_tmp044 = df_sec_incstmt_tmp043.loc[0:n_rcd_inc04-1,:].copy()

        #skip if zero income data
        n_rcd_inc_act04 = len(df_sec_incstmt_tmp044)
        if n_rcd_inc_act04 >= 1:
            #estimate report dates
            df_sec_incstmt_tmp044.loc[:,'reportedDate'] = df_sec_incstmt_tmp044.loc[:,'fiscalDateEnding'] + pd.Timedelta(fiscrpt_diff_add04,unit='D')    
        
            #meta data - incstmt: list to insert
            lst_incstmt_insrt04 = []
            lst_incstmt_insrt04 = [ [ rcd043[0], mkt_cap04, 0, rcd043[9].year,
                rcd043[9], dttm.date(2000,1,1), rcd043[1], dttm.date(2000,1,1),
                dttm.date(2000,1,1), dttm.date(2000,1,1), 1, 0 ] 
                for idx043,rcd043 in df_sec_incstmt_tmp044.iterrows() ]       

            #meta data - incstmt: next report, fiscal date
            #union
            lst_df_041 = [ df_sec_earn_tmp04.loc[:,['fiscalDateEnding','reportedDate']], 
                df_sec_incstmt_tmp044.loc[:,['fiscalDateEnding','reportedDate']] ]
            df_fiscrptdate_tmp04 = pd.DataFrame()
            df_fiscrptdate_tmp04 = pd.concat(lst_df_041,ignore_index=True)
            #flag earnings records, others are estimated
            df_fiscrptdate_tmp04.loc[:,'rd_t_estflg'] = 1
            is_earn_set04 = df_fiscrptdate_tmp04.loc[:,'fiscalDateEnding'].isin(fisc_date_earn_set042)
            df_fiscrptdate_tmp04.loc[is_earn_set04,'rd_t_estflg'] = 0
            df_fiscrptdate_tmp04.sort_values(by='fiscalDateEnding',ascending=False,inplace=True)
            df_fiscrptdate_tmp04.reset_index(drop=True,inplace=True)

            #loop records, lookup 'fiscalDateEnding', get ( 'reportedDate', 'rpt_date_t_estflg' ) at index-1
            for idx044,val044 in enumerate(lst_incstmt_insrt04):
                is_fd_t_044 = df_fiscrptdate_tmp04.loc[:,'fiscalDateEnding'] == val044[6]
                idx_fd_t_044 = df_fiscrptdate_tmp04.loc[is_fd_t_044,'fiscalDateEnding'].index[0]
                idx_fd_tp1_044 = idx_fd_t_044 - 1
                lst_incstmt_insrt04[idx044][5] = df_fiscrptdate_tmp04.loc[idx_fd_tp1_044,'reportedDate'] 
                lst_incstmt_insrt04[idx044][7] = df_fiscrptdate_tmp04.loc[idx_fd_tp1_044,'fiscalDateEnding'] 
                lst_incstmt_insrt04[idx044][11] = df_fiscrptdate_tmp04.loc[idx_fd_tp1_044,'rd_t_estflg']

            #meta data - incstmt: df from list
            df_meta_incstmt_insrt04 = pd.DataFrame(lst_incstmt_insrt04)
            df_meta_incstmt_insrt04.columns = list(df_sec_meta04.columns) 
            df_meta_insrt04 = pd.concat([df_meta_insrt04,df_meta_incstmt_insrt04],ignore_index=True) 
            df_meta_insrt04.sort_values(by='fisc_date_t',ascending=False,inplace=True)
            df_meta_insrt04.reset_index(drop=True,inplace=True)
            print('fyi:',ct04,tickn04,row04[4],np.str(n_rcd_inc_act04) + ' income records')
        else:
            print('fyi:',ct04,tickn04,row04[4],'zero income records')
            
    #meta data: rpt_seq rank by rpt_date_t ASC
    df_meta_insrt04.loc[:,'rpt_seq'] = df_meta_insrt04.loc[:,'rpt_date_t'].rank(axis=0,ascending=True)
    
    #get price eod data
    df_res_obj04 = av_api.call_av_api(tickn04,apifunc04,otptsz04,API_KEY_AV,attr_lst04,tmout04,sleepmin04,ntry04)
    
    #security dataframe
    if df_res_obj04[0] == 0:
        resp_ct04 = resp_ct04 + 1
        #format response dataframe
        df_resp04 = df_res_obj04[1].copy()
        if apifunc04 == 'TIME_SERIES_DAILY_ADJUSTED':
            df_resp04.reset_index(inplace=True)
            df_resp04.rename(columns={'index':'eod_date', '1. open':'open', '2. high':'high', '3. low':'low', 
                '4. close':'close', '5. adjusted close':'close_adj', '6. volume':'vol', '7. dividend amount':'div_amt',
                '8. split coefficient':'split_coeff', 'etl_timestamp':'etl_timestamp'},inplace=True)
            df_resp04.loc[:,'eod_date'] = pd.to_datetime(df_resp04.loc[:,'eod_date'])
            df_resp04.loc[:,'etl_timestamp'] = pd.to_datetime(df_resp04.loc[:,'etl_timestamp'])
        elif apifunc04 == 'TIME_SERIES_MONTHLY_ADJUSTED':
            df_resp04.reset_index(inplace=True)
            df_resp04.rename(columns={'index':'eod_date', '1. open':'open', '2. high':'high', '3. low':'low', 
                '4. close':'close', '5. adjusted close':'close_adj', '6. volume':'vol', '7. dividend amount':'div_amt',
                'etl_timestamp':'etl_timestamp'},inplace=True)
            df_resp04.loc[:,'eod_date'] = pd.to_datetime(df_resp04.loc[:,'eod_date'])
            df_resp04.loc[:,'etl_timestamp'] = pd.to_datetime(df_resp04.loc[:,'etl_timestamp'])
            
        df_resp04.loc[:,'Symbol'] = tickn04 
        colset04 = list(df_resp04.columns)
        colset04.insert(0, 'Symbol')
        df_resp04 = df_resp04.loc[:,colset04[0:len(colset04)-1]] 
          
        #get measure dates, price eod data                          
        df_meta_insrt04.loc[:,'meas_date_t'] = df_meta_insrt04.loc[:,'rpt_date_t'].apply( 
            lambda x04: df_resp04.loc[df_resp04.loc[:,'eod_date']<x04,'eod_date'].max() )
        df_meta_insrt04.loc[:,'meas_date_tp1'] = df_meta_insrt04.loc[:,'rpt_date_tp1'].apply( 
            lambda x04: df_resp04.loc[df_resp04.loc[:,'eod_date']<x04,'eod_date'].max() )                              
        meas_date_ary04 = set(df_meta_insrt04.loc[:,'meas_date_t'].unique()).union(set(df_meta_insrt04.loc[:,'meas_date_tp1'].unique()))
        is_price_daterng04 = df_resp04.loc[:,'eod_date'].isin(meas_date_ary04)                            
        df_price_insrt04 = df_resp04.loc[is_price_daterng04,:].copy()
        df_price_insrt04.sort_values(by='eod_date',ascending=False,inplace=True)
        df_price_insrt04.reset_index(drop=True,inplace=True)
                                                 
        #save meta data
        df_sec_meta04 = pd.concat([df_sec_meta04,df_meta_insrt04],ignore_index=True)
        #save price eod data
        if resp_ct04 == 1:
            df_sec_price_eod04 = df_price_insrt04
        else:
            df_sec_price_eod04 = pd.concat([df_sec_price_eod04,df_price_insrt04],ignore_index=True)
    else:
        excp_ct04 = excp_ct04 + 1
        print('exception:',ct04,tickn04,row04[4])
        if excp_ct04 == 1:
            df_sec_price_eod_excp04 = df_res_obj04[2]
        else:
            df_sec_price_eod_excp04 = pd.concat([df_sec_price_eod_excp04,df_res_obj04[2]],ignore_index=True)  
    
    #wait sec per request 
    time.sleep(0.83)
    if ct04 % 30 == 0:
        print('iteration:',ct04,tickn04,row04[4]) 

#clock end
load_end04 = dttm.datetime.now()
load_elapsed04 = load_end04 - load_start04 
print(load_start04,load_end04,load_elapsed04)

iteration: 30 CRM TECHNOLOGY
iteration: 60 TXN MANUFACTURING
iteration: 90 PDD TRADE & SERVICES
fyi: 120 ABNB TRADE & SERVICES 1 income records
iteration: 120 ABNB TRADE & SERVICES
exception: 126 ISG FINANCE under 2 earnings records
fyi: 127 SNOW TECHNOLOGY zero income records
iteration: 150 HCA LIFE SCIENCES
iteration: 180 REGN LIFE SCIENCES
iteration: 210 TWLO TECHNOLOGY
exception: 214 CCZ TECHNOLOGY under 2 earnings records
exception: 229 PUK-P FINANCE under 2 earnings records
iteration: 240 CMG TRADE & SERVICES
fyi: 246 CPNG TRADE & SERVICES 4 income records
fyi: 249 PLTR TECHNOLOGY 2 income records
fyi: 256 RBLX TECHNOLOGY 3 income records
iteration: 270 SCCO ENERGY & TRANSPORTATION
iteration: 300 MCHP MANUFACTURING
exception: 310 AEPPL ENERGY & TRANSPORTATION under 2 earnings records
fyi: 313 DIDI TECHNOLOGY zero income records
iteration: 330 APTV MANUFACTURING
fyi: 344 U TECHNOLOGY zero income records
iteration: 360 PPG LIFE SCIENCES
fyi: 375 HOOD TECHNOLOGY 1 income records
fyi

fyi: 1628 BIGC TECHNOLOGY 1 income records
fyi: 1636 MSP TECHNOLOGY zero income records
fyi: 1639 PSTH REAL ESTATE & CONSTRUCTION zero income records
fyi: 1648 RNW TECHNOLOGY zero income records
iteration: 1650 TPTX LIFE SCIENCES
fyi: 1679 LAW TECHNOLOGY zero income records
iteration: 1680 LGIH REAL ESTATE & CONSTRUCTION
fyi: 1694 BLND TECHNOLOGY 1 income records
exception: 1700 ADVWW TRADE & SERVICES under 2 earnings records
exception: 1701 FA TRADE & SERVICES under 2 earnings records
iteration: 1710 CDNA LIFE SCIENCES
fyi: 1715 OSCR FINANCE 2 income records
fyi: 1717 OLK TECHNOLOGY 4 income records
exception: 1718 RYAN TECHNOLOGY under 2 earnings records
fyi: 1721 VRM TRADE & SERVICES zero income records
fyi: 1729 VZIO MANUFACTURING 4 income records
iteration: 1740 ITRI LIFE SCIENCES
fyi: 1746 EVCM TECHNOLOGY 1 income records
fyi: 1747 VERV TECHNOLOGY 1 income records
exception: 1750 CTKB TECHNOLOGY under 2 earnings records
fyi: 1754 RSI TRADE & SERVICES 1 income records
fyi: 1758 IA

fyi: 2504 GLUE LIFE SCIENCES 1 income records
fyi: 2505 CYXT REAL ESTATE & CONSTRUCTION zero income records
fyi: 2512 DAWN LIFE SCIENCES 2 income records
fyi: 2515 NRIX LIFE SCIENCES 1 income records
exception: 2518 APPHW LIFE SCIENCES under 2 earnings records
iteration: 2520 LOMA MANUFACTURING
fyi: 2528 SMWB TECHNOLOGY 2 income records
fyi: 2529 SNPO TECHNOLOGY 1 income records
fyi: 2537 PRLD LIFE SCIENCES 2 income records
fyi: 2546 ICVX TECHNOLOGY zero income records
fyi: 2549 GBIO LIFE SCIENCES 1 income records
iteration: 2550 CRS MANUFACTURING
exception: 2561 IMOS MANUFACTURING
fyi: 2562 MEG TRADE & SERVICES zero income records
fyi: 2568 MKFG REAL ESTATE & CONSTRUCTION zero income records
exception: 2569 KLTR TECHNOLOGY under 2 earnings records
fyi: 2574 SGHT LIFE SCIENCES 1 income records
fyi: 2578 PCT LIFE SCIENCES 1 income records
fyi: 2579 IPOF REAL ESTATE & CONSTRUCTION 1 income records
iteration: 2580 ISEE LIFE SCIENCES
fyi: 2581 IPSC LIFE SCIENCES 1 income records
fyi: 2596 

In [42]:
df_sec_meta04.info()
df_sec_meta04.head(16)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14164 entries, 0 to 14163
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ticker          14164 non-null  object        
 1   mkt_cap_202108  14164 non-null  float64       
 2   rpt_seq         14164 non-null  float64       
 3   rpt_yr          14164 non-null  int64         
 4   rpt_date_t      14164 non-null  datetime64[ns]
 5   rpt_date_tp1    14164 non-null  datetime64[ns]
 6   fisc_date_t     14164 non-null  datetime64[ns]
 7   fisc_date_tp1   14164 non-null  datetime64[ns]
 8   meas_date_t     13288 non-null  datetime64[ns]
 9   meas_date_tp1   13608 non-null  datetime64[ns]
 10  rd_t_estflg     14164 non-null  int64         
 11  rd_tp1_estflg   14164 non-null  int64         
dtypes: datetime64[ns](6), float64(2), int64(3), object(1)
memory usage: 1.3+ MB


,ticker,mkt_cap_202108,rpt_seq,rpt_yr,rpt_date_t,rpt_date_tp1,fisc_date_t,fisc_date_tp1,meas_date_t,meas_date_tp1,rd_t_estflg,rd_tp1_estflg
0,AAPL,2.462504e+12,5.0,2021,2021-04-28,2021-07-27,2021-03-31,2021-06-30,2021-04-27,2021-07-26,0,0
1,AAPL,2.462504e+12,4.0,2021,2021-01-27,2021-04-28,2020-12-31,2021-03-31,2021-01-26,2021-04-27,0,0
2,AAPL,2.462504e+12,3.0,2020,2020-10-29,2021-01-27,2020-09-30,2020-12-31,2020-10-28,2021-01-26,0,0
3,AAPL,2.462504e+12,2.0,2020,2020-07-30,2020-10-29,2020-06-30,2020-09-30,2020-07-29,2020-10-28,0,0
4,AAPL,2.462504e+12,1.0,2020,2020-04-30,2020-07-30,2020-03-31,2020-06-30,2020-04-29,2020-07-29,0,0
5,MSFT,2.222228e+12,5.0,2021,2021-04-27,2021-07-27,2021-03-31,2021-06-30,2021-04-26,2021-07-26,0,0
6,MSFT,2.222228e+12,4.0,2021,2021-01-26,2021-04-27,2020-12-31,2021-03-31,2021-01-25,2021-04-26,0,0
7,MSFT,2.222228e+12,3.0,2020,2020-10-27,2021-01-26,2020-09-30,2020-12-31,2020-10-26,2021-01-25,0,0
8,MSFT,2.222228e+12,2.0,2020,2020-07-22,2020-10-27,2020-06-30,2020-09-30,2020-07-21,2020-10-26,0,0
9,MSFT,2.222228e+12,1.0,2020,2020-04-29,2020-07-22,2020-03-31,2020-06-30,2020-04-28,2020-07-21,0,0


In [44]:
df_sec_price_eod04.info()
df_sec_price_eod04.head(16)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16136 entries, 0 to 16135
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Symbol         16136 non-null  object        
 1   eod_date       16136 non-null  datetime64[ns]
 2   open           16136 non-null  object        
 3   high           16136 non-null  object        
 4   low            16136 non-null  object        
 5   close          16136 non-null  object        
 6   close_adj      16136 non-null  object        
 7   vol            16136 non-null  object        
 8   div_amt        16136 non-null  object        
 9   split_coeff    16136 non-null  object        
 10  etl_timestamp  16136 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(9)
memory usage: 1.4+ MB


,Symbol,eod_date,open,high,low,close,close_adj,vol,div_amt,split_coeff,etl_timestamp
0,AAPL,2021-07-26,148.27,149.83,147.7,148.99,148.766046734,72434089,0.0000,1.0,2021-09-30 00:51:04.019059
1,AAPL,2021-04-27,135.01,135.41,134.11,134.39,133.961653908,66015804,0.0000,1.0,2021-09-30 00:51:04.019059
2,AAPL,2021-01-26,143.6,144.3,141.37,143.16,142.490111669,98390555,0.0000,1.0,2021-09-30 00:51:04.019059
3,AAPL,2020-10-28,115.05,115.43,111.1,111.2,110.488827011,143937823,0.0000,1.0,2021-09-30 00:51:04.019059
4,AAPL,2020-07-29,375.0,380.92,374.85,380.16,94.2582729903,22582314,0.0000,1.0,2021-09-30 00:51:04.019059
5,AAPL,2020-04-29,284.73,289.67,283.89,287.73,71.1527062818,34320204,0.0000,1.0,2021-09-30 00:51:04.019059
6,MSFT,2021-07-26,289.0,289.69,286.642,289.05,288.494306361,23176073,0.0000,1.0,2021-09-30 00:51:07.012704
7,MSFT,2021-04-26,261.66,262.435,260.17,261.55,260.447263205,19763346,0.0000,1.0,2021-09-30 00:51:07.012704
8,MSFT,2021-01-25,229.12,229.78,224.22,229.53,228.039324364,33152095,0.0000,1.0,2021-09-30 00:51:07.012704
9,MSFT,2020-10-26,213.85,216.3399,208.1,210.08,208.16337966,37111561,0.0000,1.0,2021-09-30 00:51:07.012704


In [45]:
#save
data_elmt43 = 'meta'

#json
filnm43 = '../data/sec_' + data_elmt43 + '_' + statusdate01.strftime('%Y%m%d') + '.json'

df_sec_res_43 = df_sec_meta04.copy()

df_sec_res_43.loc[:,'rpt_date_t'] = df_sec_res_43.loc[:,'rpt_date_t'].dt.strftime('%Y-%m-%d')
df_sec_res_43.loc[:,'rpt_date_tp1'] = df_sec_res_43.loc[:,'rpt_date_tp1'].dt.strftime('%Y-%m-%d')
df_sec_res_43.loc[:,'fisc_date_t'] = df_sec_res_43.loc[:,'fisc_date_t'].dt.strftime('%Y-%m-%d')
df_sec_res_43.loc[:,'fisc_date_tp1'] = df_sec_res_43.loc[:,'fisc_date_tp1'].dt.strftime('%Y-%m-%d')
df_sec_res_43.loc[:,'meas_date_t'] = df_sec_res_43.loc[:,'meas_date_t'].dt.strftime('%Y-%m-%d')
df_sec_res_43.loc[:,'meas_date_tp1'] = df_sec_res_43.loc[:,'meas_date_tp1'].dt.strftime('%Y-%m-%d')

df_sec_res_43.to_json(filnm43)

In [47]:
#save
data_elmt44 = 'price_eod'
data_elmt45 = 'price_eod_excp'

#json
filnm44 = '../data/sec_' + data_elmt44 + '_' + statusdate01.strftime('%Y%m%d') + '.json'
filnm45 = '../data/sec_' + data_elmt45 + '_' + statusdate01.strftime('%Y%m%d') + '.json'

df_sec_res_44 = df_sec_price_eod04.copy()
df_sec_res_45 = df_sec_price_eod_excp04.copy()

df_sec_res_44.loc[:,'eod_date'] = df_sec_res_44.loc[:,'eod_date'].dt.strftime('%Y-%m-%d')
df_sec_res_44.loc[:,'etl_timestamp'] = df_sec_res_44.loc[:,'etl_timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_sec_res_44.to_json(filnm44)
df_sec_res_45.to_json(filnm45)

## 6 Load: ETF Price Data

The basic idea for this project is that company financial metrics might predict future company valuation.  However, other factors beyond individual company performance can impact equity valuations.  To attempt to control for these more macro factors, I will use an equity price benchmark that measures the overall change in valuation of the equity market as a whole.  Below I capture end-of-day prices for several exchange-traded-funds (ETFs) with the idea that one of these broad-market ETFs, ticker VTI for example, can serve as a valuation benchmark for the total market.

End-of-day prices for these ETFs are captured from Alpha Vantage using API parameter TIME_SERIES_DAILY_ADJUSTED.  The Alpha Vantage API returns a JSON file.  The function av_api.call_av_api calls the API, captures the JSON string and returns a dataframe.

In [79]:
#GET BENCHMARK ETF DATA: PRICE
#clock start
load_start05 = dttm.datetime.now()

#API parameters
apifunc05 = 'TIME_SERIES_DAILY_ADJUSTED'
otptsz05 = 'full'
attr_lst05 = ['1. open','2. high','3. low','4. close','5. adjusted close',
    '6. volume','7. dividend amount', '8. split coefficient']
tmout05 = 2.85
sleepmin05 = 0.35
ntry05 = 3

#Date parameters, initialize
n_mthbk_st05 = 31
n_mthbk_ed05 = 0
refdate05 = dttm.date(2021,8,31)
end_refmth_firstday05 = refdate05.replace(day=1)
startdate05 = end_refmth_firstday05 - relativedelta(months=n_mthbk_st05)
enddate05 = end_refmth_firstday05 - dttm.timedelta(days=1) - relativedelta(months=(n_mthbk_ed05 - 1))

#Result sets
df_etf_price_eod05 = pd.DataFrame()
df_etf_price_eod_excp05 = pd.DataFrame()

#ETF list
etf_lst05 = ['VTI','IWV','SPY','IWM','IWO','IJS','IGV','XLK','XLV','VNQ','XLF','XLE','XLB','XLY','XLC','XLI','XLU','XLP']
#etf_lst05 = ['IWV']

#Loop symbols
ct05 = 0
resp_ct05 = 0 
excp_ct05 = 0 
for tickn05 in etf_lst05:
    
    #get ticker
    ct05 = ct05 + 1
    
    #get price eod data
    df_res_obj05 = av_api.call_av_api(tickn05,apifunc05,otptsz05,API_KEY_AV,attr_lst05,tmout05,sleepmin05,ntry05)
    
    #security dataframe
    if df_res_obj05[0] == 0:
        resp_ct05 = resp_ct05 + 1
        #format response dataframe
        df_resp05 = df_res_obj05[1].copy()
        if apifunc05 == 'TIME_SERIES_DAILY_ADJUSTED':
            df_resp05.reset_index(inplace=True)
            df_resp05.rename(columns={'index':'eod_date', '1. open':'open', '2. high':'high', '3. low':'low', 
                '4. close':'close', '5. adjusted close':'close_adj', '6. volume':'vol', '7. dividend amount':'div_amt',
                '8. split coefficient':'split_coeff', 'etl_timestamp':'etl_timestamp'},inplace=True)
            df_resp05.loc[:,'eod_date'] = pd.to_datetime(df_resp05.loc[:,'eod_date'])
            df_resp05.loc[:,'etl_timestamp'] = pd.to_datetime(df_resp05.loc[:,'etl_timestamp'])
        elif apifunc05 == 'TIME_SERIES_MONTHLY_ADJUSTED':
            df_resp05.reset_index(inplace=True)
            df_resp05.rename(columns={'index':'eod_date', '1. open':'open', '2. high':'high', '3. low':'low', 
                '4. close':'close', '5. adjusted close':'close_adj', '6. volume':'vol', '7. dividend amount':'div_amt',
                'etl_timestamp':'etl_timestamp'},inplace=True)
            df_resp05.loc[:,'eod_date'] = pd.to_datetime(df_resp05.loc[:,'eod_date'])
            df_resp05.loc[:,'etl_timestamp'] = pd.to_datetime(df_resp05.loc[:,'etl_timestamp'])
            
        df_resp05.loc[:,'Symbol'] = tickn05 
        colset05 = list(df_resp05.columns)
        colset05.insert(0, 'Symbol')
        df_resp05 = df_resp05.loc[:,colset05[0:len(colset05)-1]] 
          
        #get price eod data                          
        is_price_daterng05 = ( (df_resp05.loc[:,'eod_date'] >= pd.to_datetime(startdate05)) & 
            (df_resp05.loc[:,'eod_date'] <= pd.to_datetime(enddate05)) )                            
        df_price_insrt05 = df_resp05.loc[is_price_daterng05,:].copy()
        df_price_insrt05.sort_values(by='eod_date',ascending=False,inplace=True)
        df_price_insrt05.reset_index(drop=True,inplace=True)
                                                 
        #save price eod data
        if resp_ct05 == 1:
            df_etf_price_eod05 = df_price_insrt05
        else:
            df_etf_price_eod05 = pd.concat([df_etf_price_eod05,df_price_insrt05],ignore_index=True)
    else:
        excp_ct05 = excp_ct05 + 1
        print('exception:',ct05,tickn05)
        if excp_ct05 == 1:
            df_etf_price_eod_excp05 = df_res_obj05[2]
        else:
            df_etf_price_eod_excp05 = pd.concat([df_sec_price_eod_excp05,df_res_obj05[2]],ignore_index=True)  
    
    #wait sec per request 
    time.sleep(0.83)
    if ct05 % 1 == 0:
        print('iteration:',ct05,tickn05) 

#clock end
load_end05 = dttm.datetime.now()
load_elapsed05 = load_end05 - load_start05
print(load_start05,load_end05,load_elapsed05)



iteration: 1 VTI
iteration: 2 IWV
iteration: 3 SPY
iteration: 4 IWM
iteration: 5 IWO
iteration: 6 IJS
iteration: 7 IGV
iteration: 8 XLK
iteration: 9 XLV
iteration: 10 VNQ
iteration: 11 XLF
iteration: 12 XLE
iteration: 13 XLB
iteration: 14 XLY
iteration: 15 XLC
iteration: 16 XLI
iteration: 17 XLU
iteration: 18 XLP
2021-09-30 13:01:56.062205 2021-09-30 13:02:41.415084 0:00:45.352879


In [80]:
df_etf_price_eod05.info()
df_etf_price_eod05.head(30)
#df_sec_price_eod05.tail(14)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12096 entries, 0 to 12095
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Symbol         12096 non-null  object        
 1   eod_date       12096 non-null  datetime64[ns]
 2   open           12096 non-null  object        
 3   high           12096 non-null  object        
 4   low            12096 non-null  object        
 5   close          12096 non-null  object        
 6   close_adj      12096 non-null  object        
 7   vol            12096 non-null  object        
 8   div_amt        12096 non-null  object        
 9   split_coeff    12096 non-null  object        
 10  etl_timestamp  12096 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(9)
memory usage: 1.0+ MB


,Symbol,eod_date,open,high,low,close,close_adj,vol,div_amt,split_coeff,etl_timestamp
0,VTI,2021-08-31,233.56,233.63,232.77,233.17,232.436979357,3099418,0.0000,1.0,2021-09-30 13:01:58.071476
1,VTI,2021-08-30,233.22,233.965,232.8,233.61,232.875596121,3193974,0.0000,1.0,2021-09-30 13:01:58.071476
2,VTI,2021-08-27,230.8,232.935,230.674,232.73,231.998362593,3749765,0.0000,1.0,2021-09-30 13:01:58.071476
3,VTI,2021-08-26,231.62,231.789,230.16,230.25,229.526159013,3351951,0.0000,1.0,2021-09-30 13:01:58.071476
4,VTI,2021-08-25,231.39,232.02,230.98,231.8,231.07128625,2896979,0.0000,1.0,2021-09-30 13:01:58.071476
5,VTI,2021-08-24,230.86,231.41,230.67,231.14,230.413361104,2772623,0.0000,1.0,2021-09-30 13:01:58.071476
6,VTI,2021-08-23,229.07,230.74,228.78,230.32,229.595938952,3180576,0.0000,1.0,2021-09-30 13:01:58.071476
7,VTI,2021-08-20,226.32,228.185,226.02,228.11,227.392886568,2655130,0.0000,1.0,2021-09-30 13:01:58.071476
8,VTI,2021-08-19,224.72,226.82,224.4,226.08,225.369268315,3964540,0.0000,1.0,2021-09-30 13:01:58.071476
9,VTI,2021-08-18,228.11,228.825,226.06,226.25,225.538733883,3194088,0.0000,1.0,2021-09-30 13:01:58.071476


In [81]:
#save
data_elmt51 = 'etf_price_eod'
data_elmt52 = 'etf_price_eod_excp'

#json
filnm51 = '../data/' + data_elmt51 + '_' + statusdate01.strftime('%Y%m%d') + '.json'
filnm52 = '../data/' + data_elmt52 + '_' + statusdate01.strftime('%Y%m%d') + '.json'

df_sec_res_51 = df_etf_price_eod05.copy()
df_sec_res_52 = df_etf_price_eod_excp05.copy()

df_sec_res_51.loc[:,'eod_date'] = df_sec_res_51.loc[:,'eod_date'].dt.strftime('%Y-%m-%d')
df_sec_res_51.loc[:,'etl_timestamp'] = df_sec_res_51.loc[:,'etl_timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_sec_res_51.to_json(filnm51)
df_sec_res_52.to_json(filnm52)

## 7 Conclusions

The key results of this data capture notebook are:
<ol type="1">
    <li>abc</li>
    <li>abc</li>
    <li>abc</li>
</ol> 